In [ ]:
##IoT and staff prediction

In [2]:
import pandas as pd
iot = pd.read_csv("synthetic_iot_data.csv") 

In [3]:
print(iot)

                Timestamp                           Attraction    Age_Group  \
0     2024-12-02 17:13:25         Puss In Boots' Giant Journey       Senior   
1     2024-12-01 13:26:11         Puss In Boots' Giant Journey        Adult   
2     2024-11-29 07:52:28         Puss In Boots' Giant Journey        Child   
3     2024-08-01 16:39:11         Puss In Boots' Giant Journey  Young Adult   
4     2024-05-31 18:22:35                 Revenge of the Mummy        Child   
...                   ...                                  ...          ...   
4995  2024-08-03 15:21:18                 Revenge of the Mummy       Senior   
4996  2024-09-01 17:42:25                 Revenge of the Mummy         Teen   
4997  2024-09-30 08:38:57               Transformers: The Ride         Teen   
4998  2024-11-29 14:36:58  Sesame Street Spaghetti Space Chase        Adult   
4999  2024-10-31 22:14:04  Sesame Street Spaghetti Space Chase        Adult   

      Gender Loyalty_Member  Check_In_Time  Check_O

In [7]:
import pandas as pd

# Read the Excel file
file_path = "hv15401j.xls"  
data = pd.read_excel(file_path, header=6, sheet_name='月・実数')
data = data.drop(index=0)
data = data.drop(data.columns[-1], axis=1)
data.columns = ["time_code", "date", "total_sales", "admission_sales", "restaurant_sales", "total_visitors",
              "individual_visitors", "group_visitors", "total_employees", "regular_workers", "part_time_workers"]

# Display the first few rows
print(data.head())

    time_code      date total_sales admission_sales restaurant_sales  \
1  2000000101  2000年 1月       15650            8360             7290   
2  2000000202  2000年 2月       14509            7377             7132   
3  2000000303  2000年 3月       28629           14695            13934   
4  2000000404  2000年 4月       23159           12416            10743   
5  2000000505  2000年 5月       29505           16226            13279   

  total_visitors individual_visitors group_visitors total_employees  \
1        2996812             2755960         240852           22319   
2        2654913             2236941         417972           22565   
3        5272017             4458895         813122           24063   
4        4415311             3638835         776476           24486   
5        6036756             4727314        1309442           24762   

  regular_workers part_time_workers  
1            6164             16155  
2            6095             16470  
3            6635         

In [8]:
import pandas as pd

# Assume your DataFrame is named 'data'
# Define a regex pattern to match dates like "2000年1月" or "2000年 1月"
date_pattern = r'^\d{4}年\s?\d{1,2}月$'

# Create a mask to select only rows that match the date pattern
mask = data["date"].str.match(date_pattern)

# Filter the DataFrame to keep only valid date rows
data_valid = data[mask].copy()

# Remove any extra space between "年" and the month
data_valid["date"] = data_valid["date"].str.replace("年 ", "年", regex=False)

# Convert the date column to datetime using the correct format
data_valid["date"] = pd.to_datetime(data_valid["date"], format="%Y年%m月")

print(data_valid.head())

    time_code       date total_sales admission_sales restaurant_sales  \
1  2000000101 2000-01-01       15650            8360             7290   
2  2000000202 2000-02-01       14509            7377             7132   
3  2000000303 2000-03-01       28629           14695            13934   
4  2000000404 2000-04-01       23159           12416            10743   
5  2000000505 2000-05-01       29505           16226            13279   

  total_visitors individual_visitors group_visitors total_employees  \
1        2996812             2755960         240852           22319   
2        2654913             2236941         417972           22565   
3        5272017             4458895         813122           24063   
4        4415311             3638835         776476           24486   
5        6036756             4727314        1309442           24762   

  regular_workers part_time_workers  
1            6164             16155  
2            6095             16470  
3            6635   

In [9]:
iot["Timestamp"] = pd.to_datetime(iot["Timestamp"])
# Create a year-month column in attendance and data_valid (the cleaned sales/visitor dataset)
iot["year_month"] = iot["Timestamp"].dt.to_period("M")
data_valid["year_month"] = pd.to_datetime(data_valid["date"], errors="coerce").dt.to_period("M") 

# Merge attendance data with sales/visitor dataset on the date
merged_df = pd.merge(iot, data_valid, left_on="year_month", right_on="year_month", how="left")

In [14]:
import numpy as np
merged_df.head()
merged_df["staff_count"] = np.ceil(merged_df["total_employees"]/18)
merged_df["num_of_visitor"] = np.ceil(merged_df["total_visitors"]/18)

In [20]:
merged_df.head()
print(merged_df.columns)

Index(['Timestamp', 'Attraction', 'Age_Group', 'Gender', 'Loyalty_Member',
       'Check_In_Time', 'Check_Out_Time', 'Step_Count', 'Transaction_Amount',
       'Guest_Satisfaction_Score', 'Average_Queue_Time',
       'Number_of_People_in_Queue', 'Temperature', 'Rainfall', 'Humidity',
       'year_month', 'time_code', 'date', 'total_sales', 'admission_sales',
       'restaurant_sales', 'total_visitors', 'individual_visitors',
       'group_visitors', 'total_employees', 'regular_workers',
       'part_time_workers', 'staff_count', 'num_of_visitor'],
      dtype='object')


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Select categorical and numerical columns separately
categorical_cols = ["Attraction", "Age_Group", "Gender", "Loyalty_Member"]
numerical_cols = ["staff_count", "Step_Count", "Transaction_Amount", "Guest_Satisfaction_Score",
                  "Average_Queue_Time", "Number_of_People_in_Queue", "Temperature", "Rainfall", "Humidity"]

# One-Hot Encode categorical features
encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
encoded_cats = encoder.fit_transform(merged_df[categorical_cols])

# Convert to DataFrame and get column names
encoded_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out(categorical_cols))

# Reset index to match the original DataFrame
encoded_df.index = merged_df.index

# Concatenate encoded data with numerical data
X = pd.concat([merged_df[numerical_cols], encoded_df], axis=1)

# Target variable
y = merged_df["num_of_visitor"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation Metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")


c:\Users\ReginaNoel\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


MAE: 8182.43
RMSE: 17708.07
R² Score: 0.83
